# Download SWOT Pixel Cloud products from hydroweb.next and extract information of interest for your study in a Geopackage Database for future use


## Setting the spatial zone and the time period of interest
Using a geopackage layer, preliminary created with, e.g. QGIS, to limit data download and database

In [1]:
from pixcdust.downloaders.hydroweb_next import PixCDownloader
import geopandas as gpd
from datetime import datetime

/home/hysope2/SRC/preprocess_swot_pixc/pixcdust/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# reading the area of interest polygon (could have been set)
gdf_geom = gpd.read_file('/home/hysope2/STUDIES/SWOT_Panama/DATA/aoi.gpkg')

# Limiting time period
dates = (
    datetime(2023,4,6),
    datetime(2023,4,8),
)


## Let's download matching data.
This will unfortunately lead to downloading many big files (that will be removed later). This is the only way right, but the hydroweb.next team is working on improving that.

In [3]:
pixcdownloader = PixCDownloader(
    gdf_geom,
    dates,
    verbose=0,
    path_download='/tmp/pixc',
    )
pixcdownloader.search_download()

                                              geometry  nodes_count
0 0  POLYGON ((-79.78117 9.12454, -79.79207 9.12279...           27
1 0  POLYGON ((-79.57742 8.95228, -79.57639 8.94354...           10


## Now we have all necessary files, let us extract key variables within area of interest in a geopackage database.
This geopackage format is quite efficient (though not the most efficient), and may easily be visualized in, e.g., QGIS
We are using the same geodataframe to limit the data to the area of interest

In [ ]:
from pixcdust.converters.gpkg import PixCNc2GpkgConverter
from glob import glob

In [ ]:
pixc = PixCNc2GpkgConverter(
            sorted(glob(pixcdownloader.path_download+'/*/*nc')),
            "/tmp/my_awesome_pixc_gpkg.gkpkg",
            variables=['height', 'sig0', 'classification'],
            area_of_interest=gdf_geom,
            mode='o',
        )
pixc.database_from_nc()

database has been succesfully created, we can remove the raw files

In [ ]:
import shutil
shutil.rmtree('/tmp/pixc')

# Let us use our Geopackage Database
previous steps are not necessary

Now we can open this database in a GeoDataFrame, load it in, e.g., QGIS, etc.

In [ ]:
from pixcdust.readers.gpkg import PixCGpkgReader

# nb: you may specify 
pixc_read = PixCGpkgReader(
    "/tmp/my_awesome_pixc_gpkg.gpkg"
)
pixc_read.read()
pixc_read.data

Enjoy!